In [43]:
# 1. Imports & Paths
import importlib.util
import os, sys, re, pandas as pd
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed


sys.path.append(os.path.join(os.pardir, "utils"))

DATA_DIR = os.path.join(os.pardir, "data/clusters")
resume_path = os.path.join(DATA_DIR, "resumes_clustered.csv")
jobs_path   = os.path.join(DATA_DIR, "jobs_clustered.csv")


EMB_DIR = os.path.join(os.pardir, "data/embeddings")
remb_path = os.path.join(EMB_DIR, "resume_embeddings.npy")
jemb_path   = os.path.join(EMB_DIR, "job_embeddings.npy")
SAVE_DIR = os.path.join(os.pardir, "data/embeddings")

In [44]:
%load_ext autoreload
%autoreload 2
from utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
resume_df = pd.read_csv(resume_path)
job_posts_df = pd.read_csv(jobs_path)

resume_df = resume_df[["ID", "Resume_clean"]]
job_posts_df = job_posts_df[["job_text_clean"]]

resume_embeddings = np.load(remb_path)
job_embeddings = np.load(jemb_path)

In [46]:
for j_idx, job_vec in enumerate(job_embeddings):
    # Compute similarity between this job and all resumes
    sims = cosine_similarity([job_vec], resume_embeddings)[0]
    
    # Get top 5 most similar resumes
    top_idx = sims.argsort()[-5:][::-1]  # highest 5 cosine scores

    for r_idx in top_idx:
        pairs.append({
            "job_idx": j_idx,
            "resume_idx": r_idx,
            "cosine_similarity": sims[r_idx]
        })

pairs_df = pd.DataFrame(pairs)
# pairs_df.to_csv("job_resume_cosine_pairs.csv", index=False)
# print(f"✅ Saved {len(pairs_df):,} job–resume pairs")
pairs_df.head()

,resume_idx,job_idx,cosine_similarity
0,0,3321,0.833665
1,0,2856,0.813166
2,0,3160,0.808948
3,0,2403,0.802101
4,0,2937,0.790419


In [47]:
pairs_df

,resume_idx,job_idx,cosine_similarity
0,0,3321,0.833665
1,0,2856,0.813166
2,0,3160,0.808948
3,0,2403,0.802101
4,0,2937,0.790419
...,...,...,...
94135,553,5447,0.666872
94136,1929,5447,0.648839
94137,1299,5447,0.645512
94138,581,5447,0.640185


In [48]:
resumes = pd.read_csv(resume_path)
jobs= pd.read_csv(jobs_path)

In [49]:
# Merge resume text
pairs_df["resume_text"] = pairs_df["resume_idx"].apply(lambda i: resumes.loc[i, "Resume_clean"])
# Merge job text
pairs_df["job_text"] = pairs_df["job_idx"].apply(lambda j: jobs.loc[j, "job_text_clean"])


In [50]:
finetune_df = pairs_df[["resume_text", "job_text", "cosine_similarity"]].rename(
    columns={"cosine_similarity": "label"}
)
finetune_df.head()

output_path = "data_outputs/roberta_finetuning_dataset.csv"
finetune_df.to_csv(output_path, index=False)
